# Preprocessamento de Vídeos

Usando os vídeos de entrada, criar um arquivo .csv com as features extraídas de cada vídeo.

**EDIT:** Agora temos o uso da EfficientNet B0 como extratora de características e o conseguinte uso de PCA para a redução de dimensionalidade.

In [5]:
# Importing the things
# from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
# import imageio
import cv2
import os

from sklearn.decomposition import PCA 

In [6]:
# Hiperparamters

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 224

EPOCHS = 5

In [7]:
# Importando o DataFrame

data_path = "../chalearn_dataset"
data_train = pd.read_csv(f"{data_path}/train/extraversion_data.csv")
df_train = pd.DataFrame.from_dict(data_train)
df_train.rename(columns={'Unnamed: 0': 'video_name'}, inplace=True)

df_train.head()
# df_train.tail()

,video_name,extraversion
0,zEyRyTnIw5I.005.mp4,0
1,nskJh7v6v1U.004.mp4,0
2,eHcRre1YsNA.000.mp4,0
3,VuadgOz6T7s.000.mp4,0
4,7nhJXn9PI0I.001.mp4,0


In [8]:
# Extrai um número fixo de frames igualmente espaçados de um vídeo
def extract_video_frames(video_name, data_path, n=10, dim=(224, 224)):  
    video_path = data_path + video_name
    cap = cv2.VideoCapture(video_path)
    frames = []
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    jump_size = int(length // n)
    for _ in range(n):
        ret, frame = cap.read()
        frames.append(cv2.cvtColor(cv2.resize(frame, dim), cv2.COLOR_BGR2RGB))
        for __ in range(jump_size - 1):
          cap.grab()
    return np.array(frames)

In [9]:
video_name = "be0DQawtVkE.000.mp4"
video_data_path = f"{data_path}/train/"
frames = extract_video_frames(video_name,
video_data_path)

print(frames.shape)

(10, 224, 224, 3)


In [10]:
# Cria a nossa rede extratora de features

def build_feature_extractor():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    # Usa EfficientNet B0 como extratora de características da imagem que iremos processar
    features_extractor = EfficientNetB0(include_top=False, weights='imagenet')
    outputs = features_extractor(inputs)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2022-06-14 19:46:28.771012: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-06-14 19:46:28.771072: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cruzeiro
2022-06-14 19:46:28.771085: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cruzeiro
2022-06-14 19:46:28.771194: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-06-14 19:46:28.771233: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-06-14 19:46:28.771244: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.103.1 does not match DSO version 470.129.6 -- cannot find working devices in this configuration
2022-06-14 19:46:28.794346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This Tensor

In [11]:
feature_extractor.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
Total params: 4,049,571
Trainable params: 4,007,548
Non-trainable params: 42,023
_________________________________________________________________


In [12]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["extraversion"].values

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, 7, 7, 1280), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(f"[{idx}]Extraindo features de: {path}.")
        # Gather all its frames and add a batch dimension.
        frames = extract_video_frames(path, root_dir, n=MAX_SEQ_LENGTH, dim=(IMG_SIZE, IMG_SIZE))
        frames = frames[None, ...] # O que isso faz?

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, 7, 7, 1280), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()
        print("Concluído!\n")

    return frame_features, labels

In [13]:
# Extraindo as características dos nossos videos
video_data_path = f"{data_path}/train/"
train_data, train_labels = prepare_all_videos(df_train, video_data_path)

[0]Extraindo features de: zEyRyTnIw5I.005.mp4.
Concluído!

[1]Extraindo features de: nskJh7v6v1U.004.mp4.
Concluído!

[2]Extraindo features de: eHcRre1YsNA.000.mp4.
Concluído!

[3]Extraindo features de: VuadgOz6T7s.000.mp4.
Concluído!

[4]Extraindo features de: 7nhJXn9PI0I.001.mp4.
Concluído!

[5]Extraindo features de: tEQEKN07KgQ.001.mp4.
Concluído!

[6]Extraindo features de: gsleSGEZHAs.004.mp4.
Concluído!

[7]Extraindo features de: kn6I8LdQFN0.004.mp4.
Concluído!

[8]Extraindo features de: cA43Gfcg0QA.000.mp4.
Concluído!

[9]Extraindo features de: pZxqWp0e-Ik.000.mp4.
Concluído!

[10]Extraindo features de: Xfmu-7JuDGg.002.mp4.
Concluído!

[11]Extraindo features de: aFVFvJNnFt0.000.mp4.
Concluído!

[12]Extraindo features de: cgT-3CHBmKs.000.mp4.
Concluído!

[13]Extraindo features de: P_wDFLJnW-o.004.mp4.
Concluído!

[14]Extraindo features de: zFVjcb45pjI.002.mp4.
Concluído!

[15]Extraindo features de: 0n2vWTs6XwA.001.mp4.
Concluído!

[16]Extraindo features de: 6CRet5sD6xI.003.mp4.
Co

Concluído!

[137]Extraindo features de: sDWJMBZ1mVA.005.mp4.
Concluído!

[138]Extraindo features de: mxLRr3YqgMU.000.mp4.
Concluído!

[139]Extraindo features de: gCQT7zfS7sI.002.mp4.
Concluído!

[140]Extraindo features de: Dq7Uep06KOQ.005.mp4.
Concluído!

[141]Extraindo features de: Bz3Jx4QL_6Q.001.mp4.
Concluído!

[142]Extraindo features de: 6N02tjYPLh8.005.mp4.
Concluído!

[143]Extraindo features de: 6-_1-vBNl_0.002.mp4.
Concluído!

[144]Extraindo features de: xyOq5SL0Evc.002.mp4.
Concluído!

[145]Extraindo features de: c5ZnKbaNCE8.000.mp4.
Concluído!

[146]Extraindo features de: VvSk5m90xOs.000.mp4.
Concluído!

[147]Extraindo features de: eD4b8sM-Tpw.000.mp4.
Concluído!

[148]Extraindo features de: Nv0-x4K9YFI.000.mp4.
Concluído!

[149]Extraindo features de: NDC375coN1o.005.mp4.
Concluído!

[150]Extraindo features de: vYVqhg_VGP8.000.mp4.
Concluído!

[151]Extraindo features de: cWZudzeJVXg.000.mp4.
Concluído!

[152]Extraindo features de: djsoQkroHpo.001.mp4.
Concluído!

[153]Extrain

Concluído!

[272]Extraindo features de: lnawWqnGpMc.005.mp4.
Concluído!

[273]Extraindo features de: 2PSGksTyrnI.003.mp4.
Concluído!

[274]Extraindo features de: OgPC4wnLJR0.005.mp4.
Concluído!

[275]Extraindo features de: iPblZBQpafs.001.mp4.
Concluído!

[276]Extraindo features de: 81f9t2HMNas.002.mp4.
Concluído!

[277]Extraindo features de: RlaaCqle7tY.003.mp4.
Concluído!

[278]Extraindo features de: yuE25dFTn2o.000.mp4.
Concluído!

[279]Extraindo features de: c_pyBvsSlCY.004.mp4.
Concluído!

[280]Extraindo features de: 3gKpBq-1yG4.001.mp4.
Concluído!

[281]Extraindo features de: ouYYWCY__5A.000.mp4.
Concluído!

[282]Extraindo features de: XOioICna1hc.004.mp4.
Concluído!

[283]Extraindo features de: tepWDnvIuRg.005.mp4.
Concluído!

[284]Extraindo features de: nZC82V87Pm0.002.mp4.
Concluído!

[285]Extraindo features de: PYL8BGJKu20.001.mp4.
Concluído!

[286]Extraindo features de: NOrjfF0-pQ4.004.mp4.
Concluído!

[287]Extraindo features de: K7vBdKrcFKo.001.mp4.
Concluído!

[288]Extrain

Concluído!

[407]Extraindo features de: YC3X1DcnUrk.000.mp4.
Concluído!

[408]Extraindo features de: 8n6G3V3-Tk8.005.mp4.
Concluído!

[409]Extraindo features de: FM5UaJS-bhg.002.mp4.
Concluído!

[410]Extraindo features de: j3jdT1V-DeI.004.mp4.
Concluído!

[411]Extraindo features de: CqQavmWSLrI.003.mp4.
Concluído!

[412]Extraindo features de: f9GeYKAXgAQ.005.mp4.
Concluído!

[413]Extraindo features de: MOobNUpfWiY.004.mp4.
Concluído!

[414]Extraindo features de: HCkDd0UnUgo.000.mp4.
Concluído!

[415]Extraindo features de: US4PxgfKDeA.000.mp4.
Concluído!

[416]Extraindo features de: o6JCQlOC3oE.002.mp4.
Concluído!

[417]Extraindo features de: 2x8GZaBY6TM.002.mp4.
Concluído!

[418]Extraindo features de: upgv7xmOb1I.005.mp4.
Concluído!

[419]Extraindo features de: LGgx7RVdL2Y.000.mp4.
Concluído!

[420]Extraindo features de: TiQF5QduRNE.003.mp4.
Concluído!

[421]Extraindo features de: 3taD1fEPfC8.001.mp4.
Concluído!

[422]Extraindo features de: mxLRr3YqgMU.001.mp4.
Concluído!

[423]Extrain

Concluído!

[542]Extraindo features de: okl467xvgOw.001.mp4.
Concluído!

[543]Extraindo features de: sf29orxf4X0.000.mp4.
Concluído!

[544]Extraindo features de: 8BE-2l-RBjA.003.mp4.
Concluído!

[545]Extraindo features de: fsaslN7xV-w.000.mp4.
Concluído!

[546]Extraindo features de: HekcZ0Em60E.002.mp4.
Concluído!

[547]Extraindo features de: AotbiNsU85A.001.mp4.
Concluído!

[548]Extraindo features de: WuIKjBb7s4Q.005.mp4.
Concluído!

[549]Extraindo features de: 2hhqEWiv4eI.001.mp4.
Concluído!

[550]Extraindo features de: rrdSLfnU2pg.000.mp4.
Concluído!

[551]Extraindo features de: RgZTZIf8K8g.001.mp4.
Concluído!

[552]Extraindo features de: sOkGz6gEmGk.002.mp4.
Concluído!

[553]Extraindo features de: oGa05H9UXxU.002.mp4.
Concluído!

[554]Extraindo features de: 2TbU3Eg2i4A.003.mp4.
Concluído!

[555]Extraindo features de: WRz1F6DqmRQ.004.mp4.
Concluído!

[556]Extraindo features de: 75XbCb7G8kw.002.mp4.
Concluído!

[557]Extraindo features de: KnbEca4Ibwk.000.mp4.
Concluído!

[558]Extrain

...

Concluído!

[2543]Extraindo features de: T6CMGXdPUTA.001.mp4.
Concluído!

[2544]Extraindo features de: hi8A0nUHnL8.003.mp4.
Concluído!

[2545]Extraindo features de: aoRjg63uRww.004.mp4.
Concluído!

[2546]Extraindo features de: z3DO3tdrsnQ.001.mp4.
Concluído!

[2547]Extraindo features de: GMwEmLo-gfc.000.mp4.
Concluído!

[2548]Extraindo features de: cuyIFvYNLFE.000.mp4.
Concluído!

[2549]Extraindo features de: dY7jG5gS3JE.000.mp4.
Concluído!

[2550]Extraindo features de: 6oL9yyc9iJc.003.mp4.
Concluído!

[2551]Extraindo features de: _uNup91ZYw0.002.mp4.
Concluído!

[2552]Extraindo features de: rxYWgXYTATs.001.mp4.
Concluído!

[2553]Extraindo features de: 7GRC7XjGzX8.003.mp4.
Concluído!

[2554]Extraindo features de: l3RE0rN7oEg.003.mp4.
Concluído!

[2555]Extraindo features de: JoQRNgGQCgY.004.mp4.
Concluído!

[2556]Extraindo features de: kFak4VnRnRM.005.mp4.
Concluído!

[2557]Extraindo features de: mkJjaOADfj4.002.mp4.
Concluído!

[2558]Extraindo features de: 4kIHxR6s1L4.004.mp4.
Concluíd

Concluído!

[2676]Extraindo features de: oBE9f5xD9rk.005.mp4.
Concluído!

[2677]Extraindo features de: mlXZQ8dO0nQ.004.mp4.
Concluído!

[2678]Extraindo features de: xZFZ0Mvjiuo.001.mp4.
Concluído!

[2679]Extraindo features de: AQz6_ZpzGIE.003.mp4.
Concluído!

[2680]Extraindo features de: JO44XCaQGVY.004.mp4.
Concluído!

[2681]Extraindo features de: lyZZSU1ziCQ.001.mp4.
Concluído!

[2682]Extraindo features de: blAjqQ0ja8Q.005.mp4.
Concluído!

[2683]Extraindo features de: xBiI0Oo1vBw.000.mp4.
Concluído!

[2684]Extraindo features de: NhyVkwcZbTA.002.mp4.
Concluído!

[2685]Extraindo features de: cdtZKwM5NIM.004.mp4.
Concluído!

[2686]Extraindo features de: L-rmZZP_wj8.003.mp4.
Concluído!

[2687]Extraindo features de: g0PWBzU95sI.000.mp4.
Concluído!

[2688]Extraindo features de: 84Lz5a9yRZg.004.mp4.
Concluído!

[2689]Extraindo features de: d-cZdeTp0jo.000.mp4.
Concluído!

[2690]Extraindo features de: T6CMGXdPUTA.002.mp4.
Concluído!

[2691]Extraindo features de: EYbPGTIcnjs.003.mp4.
Concluíd

Concluído!

[2809]Extraindo features de: 8py6SksNsZQ.001.mp4.
Concluído!

[2810]Extraindo features de: _LWrd9t4oZw.003.mp4.
Concluído!

[2811]Extraindo features de: KGRUaRe3r7I.002.mp4.
Concluído!

[2812]Extraindo features de: n8IiQJyqjiE.000.mp4.
Concluído!

[2813]Extraindo features de: _G3kw9HwCqY.004.mp4.
Concluído!

[2814]Extraindo features de: Wby2oMliGz0.002.mp4.
Concluído!

[2815]Extraindo features de: fkDqn2muw-A.003.mp4.
Concluído!

[2816]Extraindo features de: t0AupkXkkZw.003.mp4.
Concluído!

[2817]Extraindo features de: HPEemWelY1E.002.mp4.
Concluído!

[2818]Extraindo features de: f8eU5Pc-y0g.004.mp4.
Concluído!

[2819]Extraindo features de: CdUYBux3Vn8.000.mp4.
Concluído!

[2820]Extraindo features de: b8GPFr11adA.001.mp4.
Concluído!

[2821]Extraindo features de: GQczMGrVgbc.000.mp4.
Concluído!

[2822]Extraindo features de: WATt9OOVZh8.002.mp4.
Concluído!

[2823]Extraindo features de: Vd9biSOZ5CI.003.mp4.
Concluído!

[2824]Extraindo features de: VxtZDXM0cuQ.002.mp4.
Concluíd

Concluído!

[2942]Extraindo features de: eI_7SimPnnQ.003.mp4.
Concluído!

[2943]Extraindo features de: MI_GQLbnoew.001.mp4.
Concluído!

[2944]Extraindo features de: be0DQawtVkE.004.mp4.
Concluído!

[2945]Extraindo features de: 9eNHxfOV2Kg.002.mp4.
Concluído!

[2946]Extraindo features de: 6uqXoPJSoPg.003.mp4.
Concluído!

[2947]Extraindo features de: lErjGxBb1Bk.002.mp4.
Concluído!

[2948]Extraindo features de: ndLtwmvPpr8.003.mp4.
Concluído!

[2949]Extraindo features de: PooFvZH6fK0.001.mp4.
Concluído!

[2950]Extraindo features de: Fco1CMgOadI.002.mp4.
Concluído!

[2951]Extraindo features de: DiaibIBisuM.003.mp4.
Concluído!

[2952]Extraindo features de: WuoQDp7pU24.003.mp4.
Concluído!

[2953]Extraindo features de: RuRn2t7iPSc.002.mp4.
Concluído!

[2954]Extraindo features de: D4TU65xbF4g.000.mp4.
Concluído!

[2955]Extraindo features de: 6V92xyqy0q4.000.mp4.
Concluído!

[2956]Extraindo features de: h2-_bQdTZYY.004.mp4.
Concluído!

[2957]Extraindo features de: uNJpPYo5Qp4.002.mp4.
Concluíd

Concluído!

[3075]Extraindo features de: 6uqXoPJSoPg.005.mp4.
Concluído!

[3076]Extraindo features de: DVh_7dO2cWY.000.mp4.
Concluído!

[3077]Extraindo features de: cgp1OzTOq1o.002.mp4.
Concluído!

[3078]Extraindo features de: LCafv-LfC3Y.000.mp4.
Concluído!

[3079]Extraindo features de: ZNRfeL8nPfw.000.mp4.
Concluído!

[3080]Extraindo features de: ksCoS_ghDfM.000.mp4.
Concluído!

[3081]Extraindo features de: J28DwJsK8Do.003.mp4.
Concluído!

[3082]Extraindo features de: h7qbP1GS50U.004.mp4.
Concluído!

[3083]Extraindo features de: 1fsFL_qNYlQ.005.mp4.
Concluído!

[3084]Extraindo features de: FogaU_yB9aY.001.mp4.
Concluído!

[3085]Extraindo features de: hspeWl-Bip0.003.mp4.
Concluído!

[3086]Extraindo features de: 2q8orkMs2Jg.000.mp4.
Concluído!

[3087]Extraindo features de: 9UUtf0_ZR14.001.mp4.
Concluído!

[3088]Extraindo features de: HUJt0I1UKzo.003.mp4.
Concluído!

[3089]Extraindo features de: fBMKlYUjA9E.003.mp4.
Concluído!

[3090]Extraindo features de: oC-qxnrmJws.003.mp4.
Concluíd

In [14]:
print(train_data.shape)

(3118, 20, 7, 7, 1280)


In [16]:
# train_data.to_csv('../feature_extraction/train_data.csv')
np.save('../train_data_pre_pca', train_data)

In [17]:
np.save('../train_labels', train_labels)

In [18]:
# Aqui já temos o conjunto de treinamento pré PCA preparado
# Rodar essa célula para recuperar as informações processadas pela EfficientNet anteiormente

# Recuperar as informações de treinamento: extração de features dos vídeos

train_data = np.load('../train_data_pre_pca.npy')
train_labels = np.load('../train_labels.npy')

print(f"train_data shape: {train_data.shape}")
print(f"train_labels shape: {train_labels.shape}")

train_data shape: (3118, 20, 7, 7, 1280)
train_labels shape: (3118,)


In [19]:
train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], -1)
print(train_data.shape)

(3118, 20, 62720)


In [23]:
# print(train_data.reshape(-1, train_data.shape[-1]))
pca = PCA(n_components=1024)
pca.fit(train_data.reshape(-1, train_data.shape[-1]))

PCA(n_components=1024)

In [27]:
train_data = pca.transform(train_data.reshape(-1, train_data.shape[-1]))
print(train_data.shape)

(62360, 1024)


In [28]:
train_data = train_data.reshape(3118, 20, 1024)
print(train_data.shape)

(3118, 20, 1024)


In [29]:
# train_data.to_csv('../feature_extraction/train_data.csv')
np.save('../train_data_with_pca', train_data)